In [ ]:
!unzip "images.zip" -d "livecell-dataset"

In [ ]:
!unzip "livecell_annotations_test.json.zip" -d "ann"
!unzip "livecell_annotations_train.json.zip" -d "ann"
!unzip "livecell_annotations_val.json.zip" -d "ann"

# Training


In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [2]:
import detectron2
from pathlib import Path
import random, cv2, os
import matplotlib.pyplot as plt
import numpy as np
import pycocotools.mask as mask_util
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.structures import polygons_to_bitmask
from detectron2.evaluation import inference_on_dataset, print_csv_format
from detectron2.utils import comm
setup_logger()

<Logger detectron2 (DEBUG)>

# Loading coco formatted data

In [3]:

dataDir=Path('livecell-dataset/images')
cfg = get_cfg()
register_coco_instances('sartoriusl_train',{}, 'ann/livecell_annotations_train.json', dataDir)
register_coco_instances('sartoriusl_val',{},'ann/livecell_annotations_val.json', dataDir)
register_coco_instances('sartoriusl_test',{}, 'ann/livecell_annotations_test.json', dataDir)
metadata = MetadataCatalog.get('sartoriusl_train')
train_ds = DatasetCatalog.get('sartoriusl_train')

[12/15 19:52:16 d2.data.datasets.coco]: Loading ann/livecell_annotations_train.json takes 37.38 seconds.
WARNING [12/15 19:52:16 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/15 19:52:17 d2.data.datasets.coco]: Loaded 3253 images in COCO format from ann/livecell_annotations_train.json


# Define evaluator
Generates lines like this in the training output: [10/27 18:31:26 d2.evaluation.testing]: copypaste: MaP IoU=0.2192638391201311

In [4]:
def polygon_to_rle(polygon, shape=(520, 704)):
    #print(polygon)
    mask = polygons_to_bitmask([np.asarray(polygon) + 0.25], shape[0], shape[1])

    rle = mask_util.encode(np.asfortranarray(mask))
    return rle

# Taken from https://www.kaggle.com/theoviel/competition-metric-map-iou
def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    enc_targs = [polygon_to_rle(enc_targ[0]) for enc_targ in enc_targs]
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)

class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)

## Training

In [5]:
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("sartoriusl_train", "sartoriusl_test")
cfg.DATASETS.TEST = ("sartoriusl_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "output/model_0062620.pth"  
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.001 # Initially 30000 its were trained with 0.0001 and then remaining with 0.001
cfg.SOLVER.MAX_ITER = 8000
cfg.SOLVER.STEPS = []       
cfg.SOLVER.CHECKPOINT_PERIOD = (len(DatasetCatalog.get('sartoriusl_train')) + len(DatasetCatalog.get('sartoriusl_test'))) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
cfg.INPUT.MIN_SIZE_TRAIN = (440, 480, 520, 580, 620) # instead of (640, 672, 704, 736, 768, 800)
cfg.TEST.EVAL_PERIOD = (len(DatasetCatalog.get('sartoriusl_train')) + len(DatasetCatalog.get('sartoriusl_test'))) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
print(cfg.OUTPUT_DIR)
trainer = Trainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[12/15 19:52:46 d2.data.datasets.coco]: Loading ann/livecell_annotations_train.json takes 22.67 seconds.
WARNING [12/15 19:52:46 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/15 19:52:47 d2.data.datasets.coco]: Loaded 3253 images in COCO format from ann/livecell_annotations_train.json
[12/15 19:53:42 d2.data.datasets.coco]: Loading ann/livecell_annotations_test.json takes 47.54 seconds.
WARNING [12/15 19:53:42 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/15 19:53:43 d2.data.datasets.coco]: Loaded 1564 images in COCO format from ann/livecell_annotations_test.json
[12/15 19:54:09 d2.data.datasets.coco]: Loading ann/livecell_annotations_train.json takes 24.29 seconds.
WARNING [12/15 19:54:09 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/15 19:54:10 d2.data.datasets.coco]: Loa

In [ ]:
!ls ./output/model_final.pth